In [1]:
import data_preprocessing as dp
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, Dense, GlobalMaxPooling1D, Dropout
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
train_data, test_data, train_labels, test_labels = dp.load_imdb_data()

100%|██████████| 12500/12500 [01:05<00:00, 190.73it/s]


In [3]:
%%time
train_tokens, test_tokens = dp.process_data(train_data), dp.process_data(test_data)

CPU times: user 31.4 s, sys: 340 ms, total: 31.8 s
Wall time: 31.9 s


In [4]:
embd = dp.load_embedding_matrix('embeddings')

In [5]:
data_generator = dp.generate_batches(train_tokens, train_labels, 101, embd, 140)

In [39]:
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, padding='valid', input_shape=(140, 100)))
model.add(Dropout(0.2))
model.add(Conv1D(filters=256, kernel_size=3, padding='valid'))
model.add(GlobalMaxPooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [40]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 138, 256)          77056     
_________________________________________________________________
dropout_3 (Dropout)          (None, 138, 256)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 136, 256)          196864    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total para

In [41]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
for epoch in range(2):
    data_generator = dp.generate_batches(train_tokens, train_labels, 100, embd, 140)
    for data, labels in tqdm(data_generator):
        model.train_on_batch(data, labels)

250it [02:13,  1.80it/s]
250it [02:16,  1.81it/s]


In [17]:
data_generator = dp.generate_batches(train_tokens[:10], train_labels[:10], 2, max_seq_length=140)

In [18]:
res = []
for i in data_generator:
    res.append(i)

In [32]:
len(test_tokens)

25000

In [46]:
data_generator = dp.generate_batches(test_tokens, test_labels, 100, embd, 140)

score = model.evaluate_generator(data_generator, 241)

Exception in thread Thread-23:
Traceback (most recent call last):
  File "/usr/lib64/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib64/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/site-packages/keras/utils/data_utils.py", line 560, in data_generator_task
    generator_output = next(self._generator)
StopIteration



In [ ]:
score